# 🎈 Super Simple QTFT (Kid Edition) — with Side‑by‑Side Compare

In [ ]:
PAST_STEPS=10; FUTURE_STEPS=5; EPOCHS=30; LR=0.05; NOISE=0.2; N_SAMPLES=260; D_MODEL=16; LSTM_HIDDEN=8; PRINT_EVERY=5

In [ ]:
import math, random, numpy as np, matplotlib.pyplot as plt, torch, torch.nn as nn, torch.nn.functional as F
SEED=1234; random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'); print('Using device:',device)

In [ ]:
def make_synthetic(n=260,noise=0.2):
    t=np.arange(n); base=10+2*np.sin(2*np.pi*t/30)+0.7*np.sin(2*np.pi*t/7)
    y=base+noise*np.random.randn(n)
    Open=y+0.05*np.random.randn(n); High=y+np.abs(0.2*np.random.randn(n))
    Low=y-np.abs(0.2*np.random.randn(n)); Last=y+0.03*np.random.randn(n); Close=y+0.04*np.random.randn(n)
    return np.stack([Open,High,Low,Last],axis=1).astype(np.float32), Close.astype(np.float32)
X,y=make_synthetic(N_SAMPLES,NOISE); plt.plot(y); plt.title('Target'); plt.show()

In [ ]:
def minmax_scale(a): mn,mx=a.min(),a.max(); return ((a-mn)/(mx-mn+1e-8),(mn,mx))
def minmax_unscale(a,mm): mn,mx=mm; return a*(mx-mn)+mn
X_scaled=X.copy().astype(np.float32); mm_X=[]
for c in range(X.shape[1]): X_scaled[:,c],mmc=minmax_scale(X[:,c]); mm_X.append(mmc)
y_scaled,mm_y=minmax_scale(y.astype(np.float32))
def build_windows(Xa,ya,past,future):
    Xp,Xf,Y=[],[],[]
    for t in range(past,len(Xa)-future):
        Xp.append(Xa[t-past:t]); Xf.append(Xa[t:t+future]); Y.append(ya[t:t+future])
    return np.stack(Xp).astype(np.float32),np.stack(Xf).astype(np.float32),np.stack(Y).astype(np.float32)[...,None]
Xp,Xf,Y=build_windows(X_scaled,y_scaled,PAST_STEPS,FUTURE_STEPS)
cut=int(0.7*len(Xp)); Xptr,Xftr,Ytr=map(torch.tensor,[Xp[:cut],Xf[:cut],Y[:cut]]); Xpte,Xfte,Yte=map(torch.tensor,[Xp[cut:],Xf[cut:],Y[cut:]])
Xptr,Xftr,Ytr,Xpte,Xfte,Yte=[t.to(device) for t in [Xptr,Xftr,Ytr,Xpte,Xfte,Yte]]

In [ ]:
class NormalBrain(nn.Module):
    def __init__(self,n_features=4,d_model=16,lstm_hidden=8,future=5):
        super().__init__(); self.var_proj=nn.Linear(n_features,d_model); self.enc=nn.LSTM(d_model,lstm_hidden,batch_first=True)
        self.dec=nn.LSTM(d_model,lstm_hidden,batch_first=True); self.up=nn.Linear(lstm_hidden,d_model); self.head=nn.Linear(d_model,1)
    def forward(self,Xp,Xf): xp=self.var_proj(Xp); xf=self.var_proj(Xf); _,(h,c)=self.enc(xp); dec_out,_=self.dec(xf,(h,c)); return self.head(self.up(dec_out))
class MagicBox(nn.Module):
    def __init__(self,d_model=16):
        super().__init__(); self.enc=nn.Linear(d_model*2,d_model); self.mix1=nn.Linear(d_model,d_model); self.mix2=nn.Linear(d_model,d_model)
    def forward(self,x): s,c=torch.sin(x),torch.cos(x); h=torch.cat([s,c],dim=-1); h=F.elu(self.enc(h)); g=torch.sigmoid(self.mix1(h)); m=F.elu(self.mix2(h)); return torch.tanh(g*m+(1-g)*h)
class MagicBrain(nn.Module):
    def __init__(self,n_features=4,d_model=16,lstm_hidden=8,future=5):
        super().__init__(); self.var_proj=nn.Linear(n_features,d_model); self.enc=nn.LSTM(d_model,lstm_hidden,batch_first=True)
        self.dec=nn.LSTM(d_model,lstm_hidden,batch_first=True); self.up=nn.Linear(lstm_hidden,d_model); self.magic1=MagicBox(d_model); self.magic2=MagicBox(d_model); self.head=nn.Linear(d_model,1)
    def forward(self,Xp,Xf): xp=self.var_proj(Xp); xf=self.var_proj(Xf); _,(h,c)=self.enc(xp); dec_out,_=self.dec(xf,(h,c)); z=self.up(dec_out); return self.head(self.magic2(self.magic1(z)))

In [ ]:
def mae_loss(y_true,y_pred): return torch.mean(torch.abs(y_true-y_pred))
def train(model,Xptr,Xftr,Ytr,Xpte,Xfte,Yte,epochs=30,lr=0.05,print_every=5):
    model.to(device); opt=torch.optim.Adam(model.parameters(),lr=lr); tr,te=[],[]
    for ep in range(epochs):
        model.train(); opt.zero_grad(); yhat=model(Xptr,Xftr); loss=mae_loss(Ytr,yhat); loss.backward(); opt.step(); tr.append(loss.item())
        model.eval();
        with torch.no_grad(): te.append(mae_loss(Yte,model(Xpte,Xfte)).item())
        if (ep+1)%print_every==0: print(ep+1,tr[-1],te[-1])
    return tr,te,model
torch.manual_seed(5678)
c_model=NormalBrain(d_model=D_MODEL,lstm_hidden=LSTM_HIDDEN,future=FUTURE_STEPS).to(device)
m_model=MagicBrain(d_model=D_MODEL,lstm_hidden=LSTM_HIDDEN,future=FUTURE_STEPS).to(device)
tr_c,te_c,c_model=train(c_model,Xptr,Xftr,Ytr,Xpte,Xfte,Yte,epochs=EPOCHS,lr=LR,print_every=PRINT_EVERY)
tr_m,te_m,m_model=train(m_model,Xptr,Xftr,Ytr,Xpte,Xfte,Yte,epochs=EPOCHS,lr=LR,print_every=PRINT_EVERY)
plt.plot(te_c,label='normal'); plt.plot(te_m,label='magic'); plt.legend(); plt.show()
with torch.no_grad(): yhat_c=c_model(Xpte[:1],Xfte[:1]).cpu().numpy()[0,:,0]; yhat_m=m_model(Xpte[:1],Xfte[:1]).cpu().numpy()[0,:,0]
true=Yte[:1].cpu().numpy()[0,:,0]; def unscale_y(a): return minmax_unscale(a,mm_y)
plt.plot(unscale_y(true),label='true'); plt.plot(unscale_y(yhat_c),'--',label='normal'); plt.plot(unscale_y(yhat_m),':',label='magic'); plt.legend(); plt.show()